In [1]:
import cvxpy as cp
import numpy as np
import osqp
from scipy import sparse
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
import torch
import pandas as pd
from qpth.qp import QPFunction
import time
warnings.filterwarnings('ignore')

In [2]:
def osqp_interface(P,q,A,l,u):
    prob = osqp.OSQP()
    prob.setup(P, q, A, l, u,verbose = False)
    t0 = time.time()
    res = prob.solve()
    return res.x,res.y,time.time() - t0

In [57]:
ndim = 20
neq = 10
nineq = 10

In [58]:
P = np.random.random((ndim,ndim))
P = P.T@P+(0.0001*np.eye(ndim,ndim))
Pm = P.copy()
P = sparse.csc_matrix(P)
q = np.random.random(ndim)
A = np.random.random((neq,ndim))
G = np.random.random((nineq,ndim))
b = np.random.random(neq)
h = G@np.random.random(ndim)
osA = np.vstack([G,A])
osA = sparse.csc_matrix(osA)
l = np.hstack([-np.inf*np.ones(nineq),b])
u = np.hstack([h,b])

# 1.OSQP Forward

In [59]:
x_value, y_value, time_spent = osqp_interface(P,q,osA,l,u)
print('OSQP Forward Time spent:',time_spent)

OSQP Forward Time spent: 0.00022721290588378906


# 2.OSQP Backward

In [60]:
lambs = y_value[:nineq] # active set
active_set = np.argwhere(lambs>1e-8)
bG = G[active_set,:].squeeze()
bb = np.zeros(neq)
bh = np.zeros(len(active_set))
bq = np.ones(ndim)
osnewA = np.vstack([bG,A])
osnewA = sparse.csc_matrix(osnewA)
l_new = np.hstack([bh,bb])
u_new = np.hstack([bh,bb])

x_grad, y_grad, time_spent_backward = osqp_interface(P,bq,osnewA,l_new,u_new)
print('OSQP Backward Time spent:',time_spent_backward)

OSQP Backward Time spent: 0.00010752677917480469


# 3.CVXPY Backward

In [61]:
qq = cp.Parameter(ndim)
qq.value = q
x1 = cp.Variable(ndim)
prob = cp.Problem(cp.Minimize((1 / 2) * cp.quad_form(x1, P) + qq.T @ x1),
                              [G @ x1 <= h,
                               A @ x1 == b])
t3 = time.time()
prob.solve(requires_grad = True,solver='SCS')
print('CVXPY Forward Time Spent:',time.time() - t3)
t4 = time.time()
prob.backward()
print('CVXPY Backward Time Spent:',time.time() - t4)

CVXPY Forward Time Spent: 0.04103517532348633
CVXPY Backward Time Spent: 0.005269527435302734


In [62]:
qpf = QPFunction(verbose=0,maxIter=500)
tP, tq, tG, th, tA, tb = [torch.Tensor(x).unsqueeze(0).cuda() for x in [Pm, q, G, h, A, b]]
tq.requires_grad = True
t6 = time.time()
qpth_x_value = qpf(tP, tq, tG, th, tA, tb)
print('qpth Forward Time Spent:', time.time() - t6)
t7 = time.time()
qpth_x_value.backward(torch.ones(1, ndim).cuda())
print('qpth Backward Time Spent:', time.time() - t7)
qpth_x_grad = tq.grad.squeeze().cpu().numpy()
qpth_x_value = qpth_x_value.squeeze().detach().cpu().numpy()

qpth Forward Time Spent: 0.08447957038879395
qpth Backward Time Spent: 0.005026578903198242


# 4. Exact backward (Matrix inverse)

In [63]:
KKT_L1 = np.hstack([Pm,G.T,A.T])
KKT_L2 = np.hstack([np.diag(lambs)@G, np.diag(G@x_value-h),np.zeros((nineq,neq))])
KKT_L3 = np.hstack([A, np.zeros((neq,neq)),np.zeros((neq,nineq))])
KKT = np.vstack([KKT_L1,KKT_L2,KKT_L3])
t5 = time.time()
exact_bb =-(np.linalg.inv(KKT)@np.hstack([np.ones(ndim),np.zeros(nineq),np.zeros(neq)]))[:ndim]
print("Exact Backward Time Spent", time.time()-t5)

Exact Backward Time Spent 0.0010809898376464844


# 6. Accuracy Analysis

In [64]:
def cal_L2_accuracy(x_exact,x_approx):
    return np.sqrt(np.sum((x_exact - x_approx)**2))

In [65]:
print('Forward pass')
print(pd.DataFrame({'CVXPY':x1.value, 'qpth': qpth_x_value, 'BPQP': x_value}).head(5))
print('Backward pass')
print(pd.DataFrame({'CVXPY':qq.gradient, 'qpth': qpth_x_grad, 'BPQP': x_grad, 'exact': exact_bb}).head(5))

Forward pass
      CVXPY      qpth      BPQP
0 -0.087855 -0.087852 -0.087855
1  0.237989  0.237993  0.237989
2  0.197733  0.197736  0.197733
3 -0.278179 -0.278179 -0.278179
4 -0.053217 -0.053233 -0.053217
Backward pass
      CVXPY      qpth      BPQP     exact
0 -0.120536 -0.120573 -0.120540 -0.120539
1 -0.046952 -0.046952 -0.046951 -0.046951
2 -0.071246 -0.071218 -0.071227 -0.071227
3 -0.015605 -0.015658 -0.015612 -0.015612
4  0.114594  0.114665  0.114586  0.114586


In [66]:
print('OSQP vs CVXPY forward dif',cal_L2_accuracy(x_value , x1.value))
print('OSQP vs qpth forward dif',cal_L2_accuracy(x_value , qpth_x_value))
print('OSQP vs Exact backward dif',np.sqrt(np.sum((exact_bb - x_grad)**2)))
print('CVXPY vs Exact backward dif',np.sqrt(np.sum((exact_bb - qq.gradient)**2)))
print('qpth vs Exact backward dif',np.sqrt(np.sum((exact_bb-qpth_x_grad)**2)))
print('qpth vs CVXPY backward dif',np.sqrt(np.sum((qq.gradient-qpth_x_grad)**2)))

OSQP vs CVXPY forward dif 4.72351170640662e-07
OSQP vs qpth forward dif 5.050744830431107e-05
OSQP vs Exact backward dif 9.928261768325145e-07
CVXPY vs Exact backward dif 5.637992733069404e-05
qpth vs Exact backward dif 0.0003041202564301781
qpth vs CVXPY backward dif 0.00031885341118828315


# 7. Comparison with state-of-art differentiable QP optimizer

In [13]:
n_list = [10,100,500]
nconstraints_list = [5,50,250]

In [14]:
def QP_instances(ndim,neq,nineq):

    P = np.random.random((ndim,ndim))
    P = P.T@P+(0.001*np.eye(ndim,ndim))
    Pm = P.copy()
    P = sparse.csc_matrix(P)
    q = np.random.random(ndim)
    A = np.random.random((neq,ndim))
    G = np.random.random((nineq,ndim))
    b = np.random.random(neq)
    h = G@np.random.random(ndim)
    osA = np.vstack([G,A])
    osA = sparse.csc_matrix(osA)
    l = np.hstack([-np.inf*h,b])
    u = np.hstack([h,b])
    return P,q,G,b,A,h,osA,l,u,Pm

def QP_OSQP_backward(y_value,P,G,A):
    nineq,ndim = G.shape
    neq = A.shape[0]
    lambs = y_value[:nineq] # active set
    active_set = np.argwhere(lambs>1e-8)
    bG = G[active_set,:].squeeze()
    bb = np.zeros(neq)
    bh = np.zeros(len(active_set))
    bq = np.ones(ndim)
    osnewA = np.vstack([bG,A])
    osnewA = sparse.csc_matrix(osnewA)
    l_new = np.hstack([bh,bb])
    u_new = np.hstack([bh,bb])
    x_grad, y_grad, time_spent_backward = osqp_interface(P,bq,osnewA,l_new,u_new)
    return x_grad, y_grad, time_spent_backward

def QP_cvxpy_backward(P,G,A,h,b,q):
    nineq,ndim = G.shape
    neq = nineq
    qq = cp.Parameter(ndim)
    qq.value = q
    x1 = cp.Variable(ndim)
    prob = cp.Problem(cp.Minimize((1 / 2) * cp.quad_form(x1, P) + qq.T @ x1),
                                  [G @ x1 <= h,
                                   A @ x1 == b])
    t3 = time.time()
    try:
        prob.solve(requires_grad=True, solver='SCS')
        time_spent_forward = time.time() - t3
        t4 = time.time()
        prob.backward()
        time_spent_backward = time.time() - t4
        return x1.value,prob.value, qq.gradient,time_spent_forward,time_spent_backward
    except:
        return np.nan*np.zeros(ndim),np.nan,np.nan*np.zeros(ndim),np.nan,np.nan

def QP_qpth_evaluate(Pm,G,A,h,b,q):
    nineq,ndim = G.shape
    neq = nineq
    qpf = QPFunction(verbose=0,maxIter=500)
    tP, tq, tG, th, tA, tb = [torch.Tensor(x).unsqueeze(0).cuda() for x in [Pm, q, G, h, A, b]]
    tq.requires_grad = True
    try:
        t6 = time.time()
        qpth_x_value = qpf(tP, tq, tG, th, tA, tb)
        t7 = time.time()
        qpth_x_value.backward(torch.ones(1, ndim).cuda())
        qpth_x_grad = tq.grad.squeeze().cpu().numpy()
        qpth_x_value = qpth_x_value.squeeze().detach().cpu().numpy()
        return qpth_x_value,qpth_x_grad,time.time() - t6,time.time() - t7
    except:
        return np.nan*np.zeros(ndim),np.nan*np.zeros(ndim),np.nan,np.nan

def QP_cal_exact_backward(Pm,G,A,y_value,x_value,h):
    nineq,ndim = G.shape
    neq = A.shape[0]
    lambs = y_value[:nineq] # active set
    KKT_L1 = np.hstack([Pm,G.T,A.T])
    KKT_L2 = np.hstack([np.diag(lambs)@G, np.diag(G@x_value-h),np.zeros((nineq,neq))])
    KKT_L3 = np.hstack([A, np.zeros((neq,neq)),np.zeros((neq,nineq))])
    KKT = np.vstack([KKT_L1,KKT_L2,KKT_L3])
    t5 = time.time()
    exact_bb =-(np.linalg.inv(KKT)@np.hstack([np.ones(ndim),np.zeros(nineq),np.zeros(neq)]))[:ndim]
    return exact_bb,time.time()-t5

def dict_report(stats, key, value):
    if key in stats.keys():
        stats[key] = np.append(stats[key], value)
    else:
        stats[key] = value

In [15]:
stats = {}
iters = 200
for i in tqdm(range(iters)):
    for ndim,neq in zip(n_list,nconstraints_list):
        P,q,G,b,A,h,osA,l,u,Pm = QP_instances(ndim,neq,neq) # neq = nineq
        x_value, y_value, time_spent_forward_osqp = osqp_interface(P,q,osA,l,u) # OSQP Forward
        x_grad, y_grad, time_spent_backward_osqp = QP_OSQP_backward(y_value,P,G,A) # OSQP Backward
        x_cp_value,y_cp_value, x_cp_grad,time_spent_forward,time_spent_backward = QP_cvxpy_backward(P,G,A,h,b,q) # cvxpy Forward and Backward
        x_qpth_value, x_qpth_grad,time_spent_qpth_forward, time_qpth_backward= QP_qpth_evaluate(Pm,G,A,h,b,q)

        exact_bb,t_spent = QP_cal_exact_backward(Pm,G,A,y_value,x_value,h) # Exact backward
        acc_forward = cal_L2_accuracy(x_value,x_qpth_value)
        acc_osqp_bb = cal_L2_accuracy(exact_bb,x_grad)
        acc_cvxpy_bb = cal_L2_accuracy(exact_bb,x_cp_grad)
        acc_qpth_bb = cal_L2_accuracy(exact_bb,x_qpth_grad)

        dict_report(stats, 'Time OSQP Forward'+f' ndim:{ndim}'+f' neq=nineq={neq}', time_spent_forward_osqp)
        dict_report(stats, 'Time OSQP Backward'+f' ndim:{ndim}'+f' neq=nineq={neq}', time_spent_backward_osqp)
        dict_report(stats, 'Time CVXPY Forward'+f' ndim:{ndim}'+f' neq=nineq={neq}', time_spent_forward)
        dict_report(stats, 'Time CVXPY Backward'+f' ndim:{ndim}'+f' neq=nineq={neq}', time_spent_backward)
        dict_report(stats, 'Time Exact Backward'+f' ndim:{ndim}'+f' neq=nineq={neq}', t_spent)
        dict_report(stats, 'Time Qpth Backward'+f' ndim:{ndim}'+f' neq=nineq={neq}', time_qpth_backward)
        dict_report(stats, 'Time Qpth Forward'+f' ndim:{ndim}'+f' neq=nineq={neq}', time_spent_qpth_forward)
        dict_report(stats, 'Accuracy Forward'+f' ndim:{ndim}'+f' neq=nineq={neq}', acc_forward)
        dict_report(stats, 'Accuracy OSQP Backward'+f' ndim:{ndim}'+f' neq=nineq={neq}', acc_osqp_bb)
        dict_report(stats, 'Accuracy CVXPY Backward'+f' ndim:{ndim}'+f' neq=nineq={neq}', acc_cvxpy_bb)
        dict_report(stats, 'Accuracy QPTH Backward'+f' ndim:{ndim}'+f' neq=nineq={neq}', acc_qpth_bb)


 25%|█████████████████████████████████████▌                                                                                                                | 5/20 [00:35<01:48,  7.21s/it]

WARNING - large complementary slackness residual: 91740769109273172240642458838222107631455318870089646751672285790208.000000


 45%|███████████████████████████████████████████████████████████████████▌                                                                                  | 9/20 [01:03<01:18,  7.09s/it]

WARNING - large complementary slackness residual: 6931798828365205111015889960202655132606844485310273310504911677577251025178119372800.000000


 65%|████████████████████████████████████████████████████████████████████████████████████████████████▊                                                    | 13/20 [01:32<00:50,  7.14s/it]

WARNING - large complementary slackness residual: 25740899263144351484090013922006374303317402046995972974291380636967276647103878282608640.000000


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [02:21<00:00,  7.06s/it]


In [16]:
def get_results_table(results_dict):
    d = {}
    missing_methods = []
    for method in results_dict.keys():
        if method in results_dict:
            d[method] = ['{:.1e}({:.1e})'.format(np.nanmean(results_dict[method]),np.nanstd(results_dict[method]))]
        else:
            missing_methods.append(method)
    df = pd.DataFrame.from_dict(d, orient='index')
    df.index.names = ['avg']
    return df

In [17]:
df = get_results_table(stats)
df

,0
avg,
Time OSQP Forward ndim:10 neq=nineq=5,8.7e-05(2.7e-05)
Time OSQP Backward ndim:10 neq=nineq=5,4.4e-05(7.5e-06)
Time CVXPY Forward ndim:10 neq=nineq=5,4.8e-02(1.6e-02)
Time CVXPY Backward ndim:10 neq=nineq=5,3.8e-03(6.9e-04)
Time Exact Backward ndim:10 neq=nineq=5,2.9e-03(3.8e-03)
Time Qpth Backward ndim:10 neq=nineq=5,5.2e-03(1.3e-03)
Time Qpth Forward ndim:10 neq=nineq=5,1.1e-01(2.1e-02)
Accuracy Forward ndim:10 neq=nineq=5,2.0e-04(2.1e-04)
Accuracy OSQP Backward ndim:10 neq=nineq=5,9.6e-05(3.2e-04)


In [10]:
df.to_csv('./BPQP_QP_results.csv')

In [11]:
pd.DataFrame(stats).to_csv('./BPQP_QP_raw_results.csv')

# 8. Large scale QPs

In [19]:
l_stats = {}
nl_list = [1000,2000,3000,4000]
ncl_list = [500,1000,2000,3000]
for ndim,neq in zip(nl_list,ncl_list):
    print(f'ndim:{ndim}, neq=nineq:{neq}')
    P,q,G,b,A,h,osA,l,u,Pm = QP_instances(ndim,neq,neq) # neq = nineq
    x_value, y_value, time_spent_forward_osqp = osqp_interface(P,q,osA,l,u) # OSQP Forward
    x_grad, y_grad, time_spent_backward_osqp = QP_OSQP_backward(y_value,P,G,A) # OSQP Backward

    x_qpth_value, x_qpth_grad,time_spent_qpth_forward, time_qpth_backward= QP_qpth_evaluate(Pm,G,A,h,b,q)
    exact_bb,t_spent = QP_cal_exact_backward(Pm,G,A,y_value,x_value,h) # Exact backward

    dict_report(l_stats, 'Time OSQP Forward'+f' ndim:{ndim}'+f' neq=nineq={neq}', time_spent_forward_osqp)
    dict_report(l_stats, 'Time OSQP Backward'+f' ndim:{ndim}'+f' neq=nineq={neq}', time_spent_backward_osqp)
    dict_report(l_stats, 'Time QPTH Forward'+f' ndim:{ndim}'+f' neq=nineq={neq}', time_spent_qpth_forward)
    dict_report(l_stats, 'Time QPTH Backward'+f' ndim:{ndim}'+f' neq=nineq={neq}', time_qpth_backward)
    dict_report(l_stats, 'Time Exact Backward'+f' ndim:{ndim}'+f' neq=nineq={neq}', t_spent)


ndim:1000, neq=nineq:500

--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------

ndim:2000, neq=nineq:1000

--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues

In [20]:
df_ls = get_results_table(l_stats)
df_ls

,0
avg,
Time OSQP Forward ndim:1000 neq=nineq=500,2.0e+00(0.0e+00)
Time OSQP Backward ndim:1000 neq=nineq=500,5.4e-02(0.0e+00)
Time QPTH Forward ndim:1000 neq=nineq=500,1.2e+00(0.0e+00)
Time QPTH Backward ndim:1000 neq=nineq=500,1.0e-02(0.0e+00)
Time Exact Backward ndim:1000 neq=nineq=500,8.9e-01(0.0e+00)
Time OSQP Forward ndim:2000 neq=nineq=1000,2.2e+01(0.0e+00)
Time OSQP Backward ndim:2000 neq=nineq=1000,2.9e-01(0.0e+00)
Time QPTH Forward ndim:2000 neq=nineq=1000,2.4e+00(0.0e+00)
Time QPTH Backward ndim:2000 neq=nineq=1000,1.1e-02(0.0e+00)


In [21]:
df_ls.to_csv('./Large_scale_qp.csv')

In [6]:
# QPs with bounds
def QP_Bounds_Problem(ndim,neq):
    P = np.random.random((ndim,ndim))
    P = P.T@P+(0.0001*np.eye(ndim,ndim))
    Pm = P.copy()
    P = sparse.csc_matrix(P)
    q = np.random.random(ndim)
    A = np.random.random((neq,ndim))
    G = np.vstack([np.eye(ndim,ndim),-np.eye(ndim,ndim)])
    b = np.random.random(neq)
    h = np.hstack([np.ones(ndim),np.zeros(ndim)])
    osA = np.vstack([G,A])
    osA = sparse.csc_matrix(osA)
    l = np.hstack([np.zeros(2*ndim)+1e-3, b])
    u = np.hstack([np.ones(2*ndim), b])
    return P,q,G,b,A,h,osA,l,u,Pm

In [14]:
l_stats = {}
nl_list = [200,500,1000,2000]
ncl_list = [100,250,500,1000]
for iters in tqdm(range(20)):
    for ndim,nineq in zip(nl_list,ncl_list):
        P,q,G,b,A,h,osA,l,u,Pm = QP_instances(ndim,1,nineq)
        x_value, y_value, time_spent_forward_osqp = osqp_interface(P,q,osA,l,u) # OSQP Forward
        x_grad, y_grad, time_spent_backward_osqp = QP_OSQP_backward(y_value,P,G,A) # OSQP Backward

        # x_qpth_value, x_qpth_grad,time_spent_qpth_forward, time_qpth_backward= QP_qpth_evaluate(Pm,G,A,h,b,q)
        exact_bb,t_spent = QP_cal_exact_backward(Pm,G,A,y_value,x_value,h) # Exact backward

        dict_report(l_stats, 'Time OSQP Forward'+f' ndim:{ndim}'+f' nineq={nineq}', time_spent_forward_osqp)
        dict_report(l_stats, 'Time OSQP Backward'+f' ndim:{ndim}'+f' nineq={nineq}', time_spent_backward_osqp)
        # dict_report(l_stats, 'Time QPTH Forward'+f' ndim:{ndim}'+f' neq=nineq={neq}', time_spent_qpth_forward)
        # dict_report(l_stats, 'Time QPTH Backward'+f' ndim:{ndim}'+f' neq=nineq={neq}', time_qpth_backward)
        dict_report(l_stats, 'Time Exact Backward'+f' ndim:{ndim}'+f' nineq={nineq}', t_spent)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [13:43<00:00, 41.19s/it]


In [15]:
df_ls = get_results_table(l_stats)
df_ls

,0
avg,
Time OSQP Forward ndim:200 nineq=100,3.4e-02(8.9e-03)
Time OSQP Backward ndim:200 nineq=100,1.9e-03(5.5e-04)
Time Exact Backward ndim:200 nineq=100,7.2e-01(2.2e-01)
Time OSQP Forward ndim:500 nineq=250,1.9e-01(2.5e-02)
Time OSQP Backward ndim:500 nineq=250,7.1e-03(7.6e-04)
Time Exact Backward ndim:500 nineq=250,2.2e+00(5.3e-01)
Time OSQP Forward ndim:1000 nineq=500,1.8e+00(3.3e-01)
Time OSQP Backward ndim:1000 nineq=500,2.8e-02(1.9e-03)
Time Exact Backward ndim:1000 nineq=500,3.3e+00(7.0e-01)


In [16]:
df_ls.to_csv("./Large_scale_QPs.csv")

# 9. Large scale LPs Performance

In [18]:
def LP_instances(ndim,nineq,delta):
    neq = nineq
    s0 = np.random.randn(nineq)
    lamb0 = np.maximum(-s0, 0)
    s0 = np.maximum(s0, 0)
    x0 = np.random.random(ndim) # one feasible solution

    G = np.random.random((nineq, ndim))
    A = np.random.random((neq, ndim))
    b = A@x0
    h = G @ x0 + s0
    c = -G.T @ lamb0
    delta = 1e-6
    P = np.eye(ndim,ndim)*delta
    return P,c,b,A,G,h

def LP_cvxpy_backward(P,c,A,b,G,h):
    nineq,ndim = A.shape
    qq = cp.Parameter(ndim)
    qq.value = c
    x = cp.Variable(ndim)

    prob = cp.Problem(cp.Minimize(qq.T@x),
                 [A @ x == b,
                  G @ x <= h
                  ])
    t3 = time.time()

    prob.solve(requires_grad=True, solver='SCS')
    time_spent_forward = time.time() - t3
    t4 = time.time()
    prob.backward()
    time_spent_backward = time.time() - t4
    return x.value,prob.value, qq.gradient,time_spent_forward,time_spent_backward

def LP_OSQP_forward(P,G,A,b,h):
    nineq,ndim = G.shape
    neq = A.shape[0]
    Pm = sparse.csc_matrix(P)
    osA = np.vstack([G,A])
    osA = sparse.csc_matrix(osA)
    l = np.hstack([-np.inf*np.ones(nineq),b])
    u = np.hstack([h,b])
    x_value, y_value, time_spent = osqp_interface(Pm,c,osA,l,u)

    return x_value,y_value, time_spent, np.dot(c,x_value)

def LP_OSQP_backward(y_value,P,G,A):
    nineq,ndim = G.shape
    neq = A.shape[0]
    lambs = y_value[:nineq] # active set
    active_set = np.argwhere(lambs>1e-8)
    bG = G[active_set,:].squeeze()
    bb = np.zeros(neq)
    bh = np.zeros(len(active_set))
    bq = np.ones(ndim)
    osnewA = np.vstack([bG,A])
    osnewA = sparse.csc_matrix(osnewA)
    l_new = np.hstack([bh,bb])
    u_new = np.hstack([bh,bb])
    x_grad, y_grad, time_spent_backward = osqp_interface(P,bq,osnewA,l_new,u_new)
    return x_grad, y_grad, time_spent_backward

def LP_qpth_evaluate(Pm, G, A, h, b, q):
    nineq, ndim = G.shape
    neq = A.shape[0]
    qpf = QPFunction(verbose=0, maxIter=4000)
    tP, tq, tG, th, tA, tb = [torch.Tensor(x).unsqueeze(0).cuda() for x in [Pm, q, G, h, A, b]]
    tq.requires_grad = True

    t6 = time.time()
    qpth_x_value = qpf(tP, tq, tG, th, tA, tb)
    t7 = time.time()
    qpth_x_value.backward(torch.ones(1, ndim).cuda())
    qpth_x_grad = tq.grad.squeeze().cpu().numpy()
    qpth_x_value = qpth_x_value.squeeze().detach().cpu().numpy()
    return qpth_x_value, qpth_x_grad, time.time() - t6, time.time() - t7


def LP_cal_exact_backward(Pm,G,A,y_value,x_value,h):
    nineq,ndim = G.shape
    neq = A.shape[0]
    lambs = y_value[:nineq] # active set
    KKT_L1 = np.hstack([Pm,G.T,A.T])
    KKT_L2 = np.hstack([np.diag(lambs)@G, np.diag(G@x_value-h),np.zeros((nineq,neq))])
    KKT_L3 = np.hstack([A, np.zeros((neq,neq)),np.zeros((neq,nineq))])
    KKT = np.vstack([KKT_L1,KKT_L2,KKT_L3])
    t5 = time.time()
    exact_bb =-(np.linalg.inv(KKT)@np.hstack([np.ones(ndim),np.zeros(nineq),np.zeros(neq)]))[:ndim]
    return exact_bb,time.time()-t5

In [19]:
n_list = [10, 50, 100, 500]
nconstraints_list = [5, 10, 20, 100]
delta = 1e-6
lp_stats = {}
iters = 200
for i in tqdm(range(iters)):
    for ndim, neq in zip(n_list, nconstraints_list):
        P,c,b,A,G,h = LP_instances(ndim, neq,delta)  # neq = nineq
        Pm = sparse.csc_matrix(P)
        x_value,y_value, time_spent, f_value = LP_OSQP_forward(Pm,G,A,b,h) # OSQP Forward

        x_cp_value, y_cp_value, x_cp_grad, time_spent_forward, time_spent_backward = LP_cvxpy_backward(P,c,A,b,G,h)  # cvxpy Forward and Backward
        x_grad, y_grad, time_spent_backward_osqp = LP_OSQP_backward(y_value,Pm,G,A)  # OSQP Backward
        x_qpth_value, x_qpth_grad,time_spent_qpth_forward, time_qpth_backward=LP_qpth_evaluate(P, G, A, h, b, c)
        exact_bb, t_spent = LP_cal_exact_backward(P,G,A,y_value,x_value,h)  # Exact backward

        acc_osqp_bb = cal_L2_accuracy(exact_bb, x_grad)
        acc_cvxpy_bb = cal_L2_accuracy(exact_bb, x_cp_grad)
        acc_qpth_bb = cal_L2_accuracy(exact_bb, x_qpth_grad)

        dict_report(lp_stats, 'Time OSQP Forward' + f' ndim:{ndim}' + f' neq=nineq={neq}', time_spent)
        dict_report(lp_stats, 'Time OSQP Backward' + f' ndim:{ndim}' + f' neq=nineq={neq}', time_spent_backward_osqp)
        dict_report(lp_stats, 'Time CVXPY Forward' + f' ndim:{ndim}' + f' neq=nineq={neq}', time_spent_forward)
        dict_report(lp_stats, 'Time CVXPY Backward' + f' ndim:{ndim}' + f' neq=nineq={neq}', time_spent_backward)
        dict_report(lp_stats, 'Time QPTH Forward' + f' ndim:{ndim}' + f' neq=nineq={neq}', time_spent_qpth_forward)
        dict_report(lp_stats, 'Time QPTH Backward' + f' ndim:{ndim}' + f' neq=nineq={neq}', time_qpth_backward)
        dict_report(lp_stats, 'Time Exact Backward' + f' ndim:{ndim}' + f' neq=nineq={neq}', t_spent)

        dict_report(lp_stats, 'Accuracy OSQP Forward' + f' ndim:{ndim}' + f' neq=nineq={neq}', f_value)
        dict_report(lp_stats, 'Accuracy CVXPY Forward' + f' ndim:{ndim}' + f' neq=nineq={neq}', np.dot(c,x_cp_value))
        dict_report(lp_stats, 'Accuracy QPTH Forward' + f' ndim:{ndim}' + f' neq=nineq={neq}', np.dot(c,x_qpth_value))
        dict_report(lp_stats, 'Accuracy OSQP Backward' + f' ndim:{ndim}' + f' neq=nineq={neq}', acc_osqp_bb)
        dict_report(lp_stats, 'Accuracy CVXPY Backward' + f' ndim:{ndim}' + f' neq=nineq={neq}', acc_cvxpy_bb)
        dict_report(lp_stats, 'Accuracy QPTH Backward' + f' ndim:{ndim}' + f' neq=nineq={neq}', acc_qpth_bb)
    if i>1:
        pd.DataFrame(lp_stats).to_csv('~/PONet/solver/BPQP_LP_raw_results(2).csv')


 17%|██████████████████▏                                                                                        | 34/200 [02:20<11:52,  4.29s/it]


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------



 18%|███████████████████▊                                                                                       | 37/200 [02:34<12:03,  4.44s/it]


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------



 30%|████████████████████████████████                                                                           | 60/200 [04:17<10:12,  4.37s/it]

...

 96%|██████████████████████████████████████████████████████████████████████████████████████████████████████▎   | 193/200 [14:17<00:31,  4.50s/it]


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------



 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 197/200 [14:35<00:13,  4.46s/it]


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [14:48<00:00,  4.44s/it]


In [20]:
df_lp = get_results_table(lp_stats)
df_lp

,0
avg,
Time OSQP Forward ndim:10 neq=nineq=5,1.1e-04(7.1e-05)
Time OSQP Backward ndim:10 neq=nineq=5,1.3e-04(8.4e-04)
Time CVXPY Forward ndim:10 neq=nineq=5,2.4e-02(7.7e-03)
Time CVXPY Backward ndim:10 neq=nineq=5,4.3e-03(2.8e-03)
Time QPTH Forward ndim:10 neq=nineq=5,1.1e-01(2.4e-02)
Time QPTH Backward ndim:10 neq=nineq=5,3.9e-03(1.1e-03)
Time Exact Backward ndim:10 neq=nineq=5,1.2e-03(2.0e-03)
Accuracy OSQP Forward ndim:10 neq=nineq=5,-4.7e+00(3.6e+00)
Accuracy CVXPY Forward ndim:10 neq=nineq=5,-4.7e+00(3.6e+00)


In [13]:
df_lp.to_csv('./BPQP_LP_results.csv')

In [ ]:
pd.DataFrame(lp_stats).to_csv('./BPQP_LP_raw_results.csv')

# 10.SOCP Robust Linear Program

In [50]:
m = 3
n = 5
p = 5
n_i = 15
np.random.seed(2)
f = cp.Parameter(n)
f.value = np.random.randn(n)
# f = np.random.randn(n)
x0 = np.random.randn(n)
A = np.random.randn(n_i, n)
b = np.random.randn(n_i)
c = np.random.randn(n)
d = np.linalg.norm(A @ x0 + b, 2) - c.T @ x0

In [51]:
F = np.random.randn(p, n)
g = F @ x0

In [52]:
x = cp.Variable(n)
soc_constraints = [cp.SOC(c.T @ x + d, A@ x + b)]
prob = cp.Problem(cp.Minimize(f.T@x),
                  soc_constraints + [F @ x == g])
prob.solve(requires_grad = True)
print("The optimal value is", prob.value)
print("A solution x is")
print(x.value)
print('gradient is')
prob.backward()
print(f.gradient)

The optimal value is 2.8776099618919417
A solution x is
[-0.84175684  0.50288766 -1.24528215 -1.05795678 -0.90901835]
gradient is
[-3.33423012e-07  2.06106654e-07  2.84859605e-07 -1.55519387e-07
 -3.86698031e-07]


In [54]:
c.T @ x.value + d

8.587580824462243

In [55]:
A@ x.value + b

array([-0.05501112,  1.17923942,  2.91294843,  2.53816407,  4.2754678 ,
        0.04802842,  0.37854054,  1.53192459,  0.52552985, -2.30544176,
        2.88630956,  0.57018352,  2.15993051, -0.27068942, -4.20334623])